# Llama Stack RAG Lifecycle

In this notebook, we will walk through the lifecycle of building and evaluating a RAG pipeline using Llama Stack. 

**Example: Torchtune Knowledge Agent** 

Throughout this notebook, we will build a knowledge agent that can answer questions about the Torchtune project. 

## 0. Setup

In [13]:
from llama_stack_client import LlamaStackClient, Agent
from llama_stack.core.library_client import LlamaStackAsLibraryClient
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os
try:
    from google.colab import userdata
    os.environ['FIREWORKS_API_KEY'] = userdata.get('FIREWORKS_API_KEY')
except ImportError:
    print("Not in Google Colab environment")

# client = LlamaStackAsLibraryClient("fireworks", provider_data = {"fireworks_api_key": os.environ['FIREWORKS_API_KEY']})
# _ = client.initialize()

# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

Not in Google Colab environment


## 1. Simple Vanilla Agent

First, we will build a simple vanilla agent without any access to external knowledge base or tools, and check how it performs on a couple of questions. 


In [14]:
# First, let's come up with a couple of examples to test the agent
examples = [
    {
        "input_query": "What precision formats does torchtune support?",
        "expected_answer": "Torchtune supports two data types for precision: fp32 (full-precision) which uses 4 bytes per model and optimizer parameter, and bfloat16 (half-precision) which uses 2 bytes per model and optimizer parameter."
    },
    {
        "input_query": "What does DoRA stand for in torchtune?",
        "expected_answer": "Weight-Decomposed Low-Rank Adaptation"
    },
    {
        "input_query": "How does the CPUOffloadOptimizer reduce GPU memory usage?",
        "expected_answer": "The CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It can also optionally offload gradients to CPU by using offload_gradients=True"
    },
    {
        "input_query": "How do I ensure only LoRA parameters are trainable when fine-tuning?",
        "expected_answer": "You can set only LoRA parameters to trainable using torchtune's utility functions: first fetch all LoRA parameters with lora_params = get_adapter_params(lora_model), then set them as trainable with set_trainable_params(lora_model, lora_params). The LoRA recipe handles this automatically."
    }
]

In [16]:
simple_agent = Agent(client,
                     model=MODEL_ID, 
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project.")
for example in examples:
    simple_session_id = simple_agent.create_session(session_name=f"simple_session_{uuid.uuid4()}")
    response = simple_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=simple_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats:

* Full precision (FP32)
* Mixed precision (FP16)

It may also support other formats such as INT8 and BF16 in the future, but currently, it primarily focuses on FP32 
and FP16. 

Please note that the specific precision formats supported by Torchtune may change over time, and it's always best 
to check the official documentation for the most up-to-date information.

Question: What does DoRA stand for in torchtune?

Agent Answer: In the context of the Torchtune project, DoRA stands for "Decoupled Optimizer for Reparameterized 
Architectures".

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer in the Torchtune project is designed to reduce GPU memory usage by offloading
certain computations from the GPU to the CPU. Here's how it works:

1. **Identifying offloadable operations**: The optimizer analyzes the computation graph of the model and identifies
operations that can be offloaded from the GPU to the CPU. These operations are typically those that don't require 
the massive parallel processing capabilities of the GPU, such as data preprocessing, encoding, or decoding.
2. **Offloading operations to CPU**: The optimizer offloads the identified operations to the CPU, which frees up 
GPU memory and reduces the amount of data that needs to be transferred between the GPU and CPU.
3. **Minimizing data transfer**: The optimizer minimizes the amount of data that needs to be transferred between 
the GPU and CPU by only transferring the necessary data for the offloaded operations. This reduces the overhead of 
data transfer and helps to conserve GPU memory.
4. **Optimizing CPU-GPU synchronization**: The optimizer ensures that the CPU and GPU are properly synchronized, 
which helps to prevent unnecessary memory allocations and deallocations on the GPU.
5. **Dynamic memory allocation**: The optimizer can dynamically allocate and deallocate memory on the GPU as 
needed, which helps to reduce memory fragmentation and waste.

By offloading computations to the CPU and minimizing data transfer, the CPUOffloadOptimizer can significantly 
reduce GPU memory usage, which can lead to:

* Improved model training and inference performance
* Increased batch sizes and throughput
* Reduced out-of-memory errors
* Better support for larger models and datasets

Overall, the CPUOffloadOptimizer is a powerful tool for optimizing GPU memory usage in deep learning workloads, and
can help to improve the overall performance and efficiency of the Torchtune project.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA (Low-Rank Adaptation) parameters are trainable when fine-tuning a model with 
Torchtune, you can follow these steps:

1. **Freeze the original model weights**: Before fine-tuning, you need to freeze the original model weights to 
prevent them from being updated during the fine-tuning process. You can do this by setting the `requires_grad` 
attribute of the model parameters to `False`. This will prevent the original model weights from being updated.

2. **Create LoRA parameters**: Create LoRA parameters for the layers you want to fine-tune. LoRA parameters are 
typically added to the original model weights to adapt the model to the new task.

3. **Set LoRA parameters as trainable**: Set the LoRA parameters as trainable by setting their `requires_grad` 
attribute to `True`. This will allow the LoRA parameters to be updated during the fine-tuning process.

Here's a sample code snippet to illustrate this:
```python
import torch
import torch.nn as nn

# Assume 'model' is your pre-trained model
model = ...

# Freeze the original model weights
for param in model.parameters():
    param.requires_grad = False

# Create LoRA parameters
lora_params = []
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):  # or any other module you want to fine-tune
        lora_param = nn.Parameter(torch.randn(module.weight.shape))
        lora_params.append(lora_param)
        setattr(model, f"{name}_lora", lora_param)

# Set LoRA parameters as trainable
for param in lora_params:
    param.requires_grad = True

# Fine-tune the model with LoRA parameters
optimizer = torch.optim.Adam(lora_params, lr=1e-4)
```
By following these steps, you can ensure that only the LoRA parameters are trainable during fine-tuning, while 
keeping the original model weights frozen.

#### 1.1 Evaluate Agent Responses
Let's gather up the agent's logs and evaluate the agent's performance. We can see that our agent's response is quite bad and off from the expected answer.

In [17]:
eval_rows = []
for i, session_id in enumerate(simple_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=simple_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.3}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. **Expert Answer**: The expert states that Torchtune supports two precision formats: fp32 (full-precision) and bfloat16 (half-precision).\n\n2. **Submitted Answer**: The submission mentions that Torchtune supports full precision (FP32) and mixed precision (FP16). It also speculates about potential future support for other formats like INT8 and BF16, but emphasizes the current focus on FP32 and FP16.\n\n3. **Comparison**:\n   - Both answers agree on the support for FP32.\n   - The expert mentions bfloat16 (BF16), while the submission mentions FP16 and speculates about BF16 in the future. This is a key difference as the expert confirms BF16 support, whereas the submission does not.\n   - The submission introduces FP16, which is not mentioned by the expert.\n   - The submission also speculates about future support for INT8 and BF16, which is not addressed by the expert.\n\n4. **Conclusion**: There is a disagreement between the submitted answer and the expert answer regarding the precision formats supported by Torchtune. The expert confirms BF16 support, while the submission does not, and instead mentions FP16, which the expert does not confirm. Therefore, the correct choice is (D).'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation".\n2. The submitted answer states that DoRA stands for "Decoupled Optimizer for Reparameterized Architectures".\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer regarding what DoRA stands for in the context of torchtune.\n5. Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on the CPU and performing optimizer steps on the CPU. It also mentions the optional offloading of gradients to the CPU.\n2. The submitted answer describes a broader mechanism of offloading computations from the GPU to the CPU, including identifying offloadable operations, minimizing data transfer, optimizing CPU-GPU synchronization, and dynamic memory allocation.\n3. The submitted answer does not explicitly mention keeping optimizer states on the CPU or performing optimizer steps on the CPU, which are key points in the expert answer.\n4. The submitted answer provides additional details about the process of offloading operations and its benefits, which are not mentioned in the expert answer.\n5. The submitted answer does not conflict with the expert answer but rather expands on the concept of offloading to the CPU with additional mechanisms and benefits.\n\nBased on this analysis, the submitted answer is a superset of the expert answer and is fully consistent with it, as it includes all the information from the expert answer and adds more details.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Summary**: The expert answer provides a concise method to ensure only LoRA par

## 2. Search Agent

Now, let's see how we can improve the agent's performance by adding a search tool.

In [18]:
search_agent = Agent(client, 
                     model=MODEL_ID,
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the search tool to answer questions.",
                     tools=["builtin::websearch"])
for example in examples:
    search_session_id = search_agent.create_session(session_name=f"search_session_{uuid.uuid4()}")
    response = search_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=search_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats:

* bf16 (16-bit floating-point format)
* fp32 (32-bit floating-point format, also known as "full-precision")

It's worth noting that torchtune also provides support for mixed-precision techniques, which allow for the use of 
different precision formats for different parts of the model or during different stages of training.

Question: What does DoRA stand for in torchtune?

Agent Answer: DoRA stands for "Decoupled Orthogonal Random Adaptation" in torchtune.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer reduces GPU memory usage by offloading gradients and trainable parameters to 
the CPU, allowing for more efficient use of GPU memory. This can be achieved by setting `offload_gradients=True` in
the CPUOffloadOptimizer, which frees gradients once device-to-host transfer finishes. Additionally, using paged 
Adam with `optimizer_in_bwd=True` can also help reduce memory usage. However, it's important to note that the 
actual memory usage may vary depending on the specific use case and model architecture.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA parameters are trainable when fine-tuning, you can use the `set_trainable_params`
function from the `torchtune.modules.peft.peft_utils` module. This function allows you to specify which parameters 
to make trainable, and you can use it to set only the LoRA parameters as trainable.

Here is an example of how to do this:
```
import torch
from torchtune.models.llama2 import llama2_7b, lora_llama2_7b
from torchtune.modules.peft.peft_utils import get_adapter_params, set_trainable_params

# Load the model and adapter
model = llama2_7b()
adapter = lora_llama2_7b()

# Get the adapter parameters
adapter_params = get_adapter_params(adapter)

# Set only the adapter parameters as trainable
set_trainable_params(model, adapter_params)
```
This code loads the LLaMA-2 model and the LoRA adapter, gets the adapter parameters, and then sets only those 
parameters as trainable using the `set_trainable_params` function. This ensures that only the LoRA parameters are 
updated during fine-tuning, while the rest of the model remains frozen.

#### 2.1 Evaluate Agent Responses

We can see that with a search tool, the agent's performance is much better, and have less hallucinations. 

In [19]:
eval_rows = []
for i, session_id in enumerate(search_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=search_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.44999999999999996}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. **Expert Answer Details**: The expert answer states that Torchtune supports two precision formats: fp32 (full-precision) and bfloat16 (half-precision).\n\n2. **Submitted Answer Details**: The submitted answer mentions two precision formats: bf16 (16-bit floating-point format) and fp32 (32-bit floating-point format, also known as "full-precision"). It also adds that Torchtune supports mixed-precision techniques.\n\n3. **Comparison of Precision Formats**:\n   - The expert answer uses "bfloat16" while the submitted answer uses "bf16". These are equivalent terms, as "bf16" is a common abbreviation for "bfloat16".\n   - Both answers mention "fp32" as a supported precision format.\n\n4. **Additional Information in Submission**: The submitted answer includes additional information about mixed-precision techniques, which is not mentioned in the expert answer.\n\n5. **Consistency Check**: The submitted answer includes all the information from the expert answer and adds more details about mixed-precision techniques. There is no conflict between the two answers.\n\nBased on the above analysis, the submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer states that DoRA stands for "Decoupled Orthogonal Random Adaptation."\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer regarding what DoRA stands for in torchtune.\n5. Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. **Expert Answer Analysis**: The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on the CPU and performing optimizer steps on the CPU. It also mentions the optional offloading of gradients to the CPU by setting `offload_gradients=True`.\n\n2. **Submitted Answer Analysis**: The submitted answer mentions offloading gradients and trainable parameters to the CPU, which allows for more efficient use of GPU memory. It specifies the use of `offload_gradients=True` to free gradients after device-to-host transfer. Additionally, it introduces the concept of using paged Adam with `optimizer_in_bwd=True` to help reduce memory usage. It also notes that actual memory usage may vary depending on the use case and model architecture.\n\n3. **Comparison**:\n   - Both answers mention offloading gradients to the CPU using `offload_gradients=True`.\n   - The expert answer focuses on keeping optimizer states and performing optimizer steps on the CPU, while the submitted answer expands on this by mentioning trainable parameters and the use of paged Adam.\n   - The submitted answer provides additional context about memory usage variability and the use of paged Adam, which is not mentioned in the expert answer.\n\n4. **Conclusion**: The submitted answer is a superset of the expert answer as it includes all the information from the expert answer and adds more details about trainable parameters, paged Adam, and memory usage variability. There 

### 3. RAG Agent

Now, let's see how we can improve the agent's performance by adding a RAG tool that explicitly retrieves information from Torchtune's documentation. 

In [ ]:
from llama_stack_client.types import Document
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]
documents = [
    Document(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

vector_providers = [
    provider for provider in client.providers.list() if provider.api == "vector_io"
]
selected_vector_provider = vector_providers[0]
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="nomic-embed-text-v1.5",
    embedding_dimension=768,
    provider_id=selected_vector_provider.provider_id,
)

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [27]:
rag_agent = Agent(
    client,
    model=MODEL_ID,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the RAG tool to answer questions.",
    tools=[{
        "name": "builtin::rag",
        "args": {"vector_db_ids": [vector_db_id]},
    }],
)

for example in examples:
    rag_session_id = rag_agent.create_session(session_name=f"rag_session_{uuid.uuid4()}")
    response = rag_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=rag_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports the following precision formats:

* bfloat16 (half-precision)
* fp32 (full-precision)
* int8 (integer 8-bit)
* int4 (integer 4-bit)

Note that mixed-precision training is not currently supported in torchtune.

Question: What does DoRA stand for in torchtune?

Agent Answer: DoRA stands for "Decoupled Orthogonal Random Axes" in the context of the Torchtune project.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer reduces GPU memory usage by offloading optimizer states and gradients to CPU,
thus reducing the memory usage on the GPU. This is especially useful when training large models or when using 
stateful optimizers, as it can significantly reduce the memory requirements. However, it may come at the cost of 
increased CPU RAM usage and potentially slower training speeds.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA parameters are trainable when fine-tuning, you can use the `get_adapter_params` 
and `set_trainable_params` functions from `torchtune.modules.peft.peft_utils`. 

Here is how to do it:

```python
from torchtune.modules.peft.peft_utils import get_adapter_params, set_trainable_params

# Fetch all params from the model that are associated with LoRA.
lora_params = get_adapter_params(lora_model)

# Set requires_grad=True on lora_params, and requires_grad=False on all others.
set_trainable_params(lora_model, lora_params)
```

In [28]:
eval_rows = []
for i, session_id in enumerate(rag_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.3}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that Torchtune supports two precision formats: fp32 and bfloat16.\n2. The submitted answer lists four precision formats: bfloat16, fp32, int8, and int4.\n3. The submitted answer includes the two formats mentioned by the expert (bfloat16 and fp32), but also adds int8 and int4, which are not mentioned by the expert.\n4. The submitted answer also states that mixed-precision training is not supported, which is not addressed in the expert answer.\n5. Since the submitted answer includes additional precision formats (int8 and int4) that are not mentioned by the expert, there is a factual disagreement between the two answers regarding the supported precision formats.\n6. Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.0,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'D',
│   │   │   │   │   │   'rationale': '1. The expert answer states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer states that DoRA stands for "Decoupled Orthogonal Random Axes."\n3. The two answers provide completely different expansions for the acronym DoRA.\n4. Since the expansions are different, there is a clear disagreement between the submitted answer and the expert answer.\n5. Therefore, the correct choice is (D) There is a disagreement between the submitted answer and the expert answer.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It also mentions the optional offloading of gradients to CPU using offload_gradients=True.\n2. The submitted answer states that the CPUOffloadOptimizer reduces GPU memory usage by offloading optimizer states and gradients to CPU. It also mentions that this is useful for large models or stateful optimizers and notes potential downsides like increased CPU RAM usage and slower training speeds.\n3. The submitted answer includes all the points mentioned in the expert answer: offloading optimizer states and optionally gradients to CPU.\n4. Additionally, the submitted answer provides extra context about the usefulness for large models and potential downsides, which are not mentioned in the expert answer.\n5. There is no factual disagreement between the two answers; the submitted answer simply provides more information.\n\nBased on this analysis, the submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Identify the core content of both answers:**\n   - The expert answer explains how to set only LoRA parameters as trainable using torchtune's utility functions by fetching all LoRA parameters with `get_adapter_params(lora_model)` and setting them as trainable with `set_trainable_params(lora_model, lora_params)`. It also mentions that the LoRA recipe handles this automatically.\n   - The submitted answer provides a similar explanation, detailing the use of `get_adapter_params` and `set_trainable_params` from `torchtune.modules.peft.peft_utils` to ensure only LoRA parameters are trainable. It includes a code snippet demonstrating t

#### Deep dive into RAG Tool Performance
- Now, let's take a closer look at how the RAG tool is doing, specifically on the second example where the agent's answer is not correct on identifying what DoRA stands for. 
- Notice that the issue lies with the retrieval step, where the retrieved document is not relevant to the question. 

In [29]:
session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=rag_agent.sessions[1])
pprint(session_response.turns)

[
│   Turn(
│   │   input_messages=[UserMessage(content='What does DoRA stand for in torchtune?', role='user', context=None)],
│   │   output_message=CompletionMessage(
│   │   │   content='DoRA stands for "Decoupled Orthogonal Random Axes" in the context of the Torchtune project.',
│   │   │   role='assistant',
│   │   │   stop_reason='end_of_turn',
│   │   │   tool_calls=[]
│   │   ),
│   │   session_id='b5b5b9c5-1f14-404a-9677-cdb413b9f328',
│   │   started_at=datetime.datetime(2025, 3, 7, 10, 35, 24, 235903, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
│   │   steps=[
│   │   │   InferenceStep(
│   │   │   │   api_model_response=CompletionMessage(
│   │   │   │   │   content='',
│   │   │   │   │   role='assistant',
│   │   │   │   │   stop_reason='end_of_turn',
│   │   │   │   │   tool_calls=[
│   │   │   │   │   │   ToolCall(
│   │   │   │   │   │   │   arguments={'query': 'DoRA meaning in Torchtune'},
│   │   │   │   │   │   │   call_id='c2c088b9-cf2f-41b5-a050-dd5743112f48',
│   │   │   │   │   │   │   tool_name='knowledge_search'
│   │   │   │   │   │   )
│   │   │   │   │   ]
│   │   │   │   ),
│   │   │   │   step_id='27ba55cd-0252-4cff-8141-129b3b8dd021',
│   │   │   │   step_type='inference',
│   │   │   │   turn_id='bb111412-e2e9-40ca-9cd2-87df200807ab',
│   │   │   │   completed_at=datetime.datetime(2025, 3, 7, 10, 35, 26, 226185, tzinfo=TzInfo(-08:00)),
│   │   │   │   started_at=datetime.datetime(2025, 3, 7, 10, 35, 24, 236359, tzinfo=TzInfo(-08:00))
│   │   │   ),
│   │   │   ToolExecutionStep(
│   │   │   │   step_id='e7da6bb1-a704-4a2e-9954-5d54d8a1fc5d',
│   │   │   │   step_type='tool_execution',
│   │   │   │   tool_calls=[
│   │   │   │   │   ToolCall(
│   │   │   │   │   │   arguments={'query': 'DoRA meaning in Torchtune'},
│   │   │   │   │   │   call_id='c2c088b9-cf2f-41b5-a050-dd5743112f48',
│   │   │   │   │   │   tool_name='knowledge_search'
│   │   │   │   │   )
│   │   │   │   ],
│   │   │   │   tool_responses=[
│   │   │   │   │   ToolResponse(
│   │   │   │   │   │   call_id='c2c088b9-cf2f-41b5-a050-dd5743112f48',
│   │   │   │   │   │   content=[
│   │   │   │   │   │   │   TextContentItem(
│   │   │   │   │   │   │   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   │   │   │   │   │   │   type='text'
│   │   │   │   │   │   │   ),
│   │   │   │   │   │   │   TextContentItem(
│   │   │   │   │   │   │   │   text='Result 1:\nDocument_id:num-0\nContent: etune\n:func:`torchtune.models.llama3.llama3_8b` with DoRA, you would use :func:`torchtune.models.llama3.lora_llama3_8b` with ``use_dora=True``:\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lora_single_device \\\n  model.use_dora=True\n\n.. code-block:: yaml\n\n  model:\n    _component_: torchtune.models.lora_llama3_8b\n    use_dora: True\n\nSince DoRA extends LoRA, the parameters for :ref:`customizing LoRA <glossary_lora>` are identical. You can also quantize the base model weights like in :ref:`glossary_qlora` by using ``quantize=True`` to reap\neven more memory savings!\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lora_single_device \\\n  model.apply_lora_to_mlp=True \\\n  model.lora_attn_modules=["q_proj","k_proj","v_proj"] \\\n  model.lora_rank=16 \\\n  model.lora_alpha=32 \\\n  model.use_dora=True \\\n  model.quantize_base=True\n\n.. code-block:: yaml\n\n  model:\n    _component_: torchtune.models.lora_llama3_8b\n    apply_lora_to_mlp: True\n    lora_attn_modules: ["q_proj", "k_proj", "v_proj"]\n    lora_rank: 16\n    lora_alpha: 32\n    use_dora: True\n    quantize_base: True\n\n\n.. note::\n\n   Under the hood, we\'ve enabled DoRA by adding the :class:`~torchtune.modules.peft.DoRALinear` module, which we swap\n   out for :class:`~torchtune.modules.peft.LoRALinear` when ``use_dora=True``.\n\n.. _glossary_distrib:\n\n\n.. TODO\n\n.. Distributed\n.. -----------\n\n.. .. _glossary_fsdp:

### 3.1 Improved RAG with Long Context

- Instead of performing reteival tool, we send documents as attachments to the agent and let it use the entire document context. 
- Note how that the model is able to understand the entire context from documentation and answers the question with better factuality with improved retrieval. 

In [19]:
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]

attachments = [
    {
        "content": {
            "uri": f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        },
        "mime_type": "text/plain",
    }

    for i, url in enumerate(urls)
]

rag_attachment_agent = Agent(
    client,
    model=MODEL_ID,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. Use context from attached documentation for Torchtune to answer questions.",
)

for example in examples:
    session_id = rag_attachment_agent.create_session(session_name=f"rag_attachment_session_{uuid.uuid4()}")
    response = rag_attachment_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=session_id,
        documents=attachments,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")



Question: What precision formats does torchtune support?

Agent Answer: Torchtune supports two precision formats: `fp32` (full-precision) and `bfloat16` (half-precision). 
The `bfloat16` format uses 2 bytes per model parameter, which is half the memory of `fp32`, and also improves 
training speed.

Question: What does DoRA stand for in torchtune?

Agent Answer: DoRA stands for Weight-Decomposed Low-Rank Adaptation. It is a variant of LoRA (Low-Rank Adaptation) 
that further decomposes the pre-trained weights into two components: magnitude and direction. The magnitude 
component is a scalar vector that adjusts the scale, while the direction component corresponds to the original LoRA
decomposition and updates the orientation of weights. DoRA adds a small overhead to LoRA training due to the 
addition of the magnitude parameter, but it has been shown to improve the performance of LoRA, particularly at low 
ranks.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: The CPUOffloadOptimizer reduces GPU memory usage by offloading optimizer states and gradients to the 
CPU, and performing optimizer steps on the CPU. This can significantly reduce GPU memory usage at the cost of CPU 
RAM and training speed.

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: To ensure only LoRA parameters are trainable when fine-tuning, you can use the `set_trainable_params`
function from `torchtune.modules.peft.peft_utils` to set the `requires_grad` attribute of the LoRA parameters to 
`True` and the `requires_grad` attribute of the other parameters to `False`.

Here is an example:
```python
from torchtune.modules.peft.peft_utils import get_adapter_params, set_trainable_params

# Get the LoRA parameters
lora_params = get_adapter_params(model)

# Set the LoRA parameters to trainable and the other parameters to non-trainable
set_trainable_params(model, lora_params)
```
This will ensure that only the LoRA parameters are updated during fine-tuning, while the other parameters remain 
frozen.

Alternatively, you can also use the `lora_finetune` recipe in torchtune, which automatically sets the LoRA 
parameters to trainable and the other parameters to non-trainable. You can run the recipe using the following 
command:
```bash
tune run lora_finetune --config llama2/7B_lora
```
This will fine-tune the LoRA parameters of the Llama2 model using the default settings. You can modify the config 
file to change the hyperparameters or the model architecture.

In [16]:
eval_rows = []
for i, session_id in enumerate(rag_attachment_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=rag_attachment_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

ScoringScoreResponse(
│   results={
│   │   'braintrust::factuality': ScoringResult(
│   │   │   aggregated_results={'average': {'average': 0.6}},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. Both the expert and the submitted answers mention that Torchtune supports two precision formats: `fp32` (full-precision) and `bfloat16` (half-precision).\n2. The expert answer specifies that `fp32` uses 4 bytes per model and optimizer parameter, while `bfloat16` uses 2 bytes per model and optimizer parameter.\n3. The submitted answer also mentions that `bfloat16` uses 2 bytes per model parameter, which is consistent with the expert answer.\n4. The submitted answer adds that `bfloat16` improves training speed, which is additional information not present in the expert answer.\n5. There is no conflict between the submitted answer and the expert answer; the submitted answer simply provides more information.\n\nBased on this analysis, the submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. The expert answer provides the definition of DoRA as "Weight-Decomposed Low-Rank Adaptation."\n2. The submitted answer also states that DoRA stands for "Weight-Decomposed Low-Rank Adaptation," which matches the expert answer.\n3. The submitted answer includes additional information about DoRA, explaining that it is a variant of LoRA and describing how it decomposes pre-trained weights into magnitude and direction components.\n4. The submitted answer further explains the role of the magnitude component and the direction component, and mentions the performance improvement and overhead associated with DoRA.\n5. The additional details in the submitted answer do not contradict the expert answer; instead, they expand upon it.\n6. Therefore, the submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': '1. The expert answer states that the CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It also mentions the optional offloading of gradients to CPU with the parameter offload_gradients=True.\n\n2. The submitted answer states that the CPUOffloadOptimizer reduces GPU memory usage by offloading optimizer states and gradients to the CPU, and performing optimizer steps on the CPU. It adds that this can significantly reduce GPU memory usage at the cost of CPU RAM and training speed.\n\n3. Comparing both answers:\n   - Both answers agree on offloading optimizer states to the CPU and performing optimizer steps on the CPU.\n   - Both mention the offloading of gradients to the CPU, but the expert answer specifies it as optional with a parameter, while the submission does not specify this detail.\n   - The submission adds additional information about the trade-off involving CPU RAM and training speed, which is not mentioned in the expert answer.\n\n4. The submitted answer includes all the details from the expert answer and adds more information about the trade-offs, making it a superset of the expert answer.\n\nTherefore, the correct choice is (B) The submitted answer is a superset of the expert answer and is fully consistent with it.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 0.6,
│   │   │   │   │   'metadata': {
│   │   │   │   │   │   'choice': 'B',
│   │   │   │   │   │   'rationale': "1. **Expert Answer Analysis**: The expert answer provides a method t